In [2]:
import tensorflow as tf

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST", one_hot=True)

Extracting MNIST\train-images-idx3-ubyte.gz
Extracting MNIST\train-labels-idx1-ubyte.gz
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz


In [7]:
# Constants
input_width = 28
input_height = 28
input_channels = 1
input_pixels = input_width * input_height

n_cov1 = 32
n_cov2 = 64
cov1_k = 5
cov2_k = 5
stride_cov1 = 1
stride_cov2 = 1
maxpoll_cov1 = 2
maxpoll_cov2 = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(maxpoll_cov1*maxpoll_cov2)) * (input_width//(maxpoll_cov1*maxpoll_cov2)) * n_cov2

In [20]:
weight = {
    'wc1' : tf.Variable(tf.random_normal([cov1_k,cov1_k,input_channels,n_cov1])),
    'wc2' : tf.Variable(tf.random_normal([cov2_k,cov2_k,n_cov1,n_cov2])),
    'wh1' : tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    'wo' : tf.Variable(tf.random_normal([n_hidden,n_out]))
}

bias = {
    'bc1' : tf.Variable(tf.random_normal([n_cov1])),
    'bc2' : tf.Variable(tf.random_normal([n_cov2])),
    'bh1' : tf.Variable(tf.random_normal([n_hidden])),
    'bo' : tf.Variable(tf.random_normal([n_out]))

}

In [21]:
def conv(x, weight, bias, stride = 2):
    out = tf.nn.conv2d( x, weight, padding="SAME", strides = [1,stride,stride,1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out
def max_pooling(x, k = 2):
    return tf.nn.max_pool(x, padding="SAME", ksize=[1,k,k,1], strides = [1,k,k,1])

In [23]:
# Forward propagation 

def cnn(x, weights, bias):
    x = tf.reshape(x, shape=[-1, input_width, input_height, input_channels])
    
    cov1 = covl(x, weights['wc1'], bias['bc1'], stride_cov1)
    cov_poll_1 = max_pooling(cov1, maxpoll_cov1)
    
    cov1 = covl(cov_poll_1, weights['wc2'], bias['bc2'], stride_cov1)
    cov_poll_2 = max_pooling(cov1, maxpoll_cov1)
    
    hidden_input = tf.reshape(cov_poll_2, shape=[-1, input_size_to_hidden])
    hidden_out_before_activation = tf.add(tf.maxmul(hidden_input, weight['wh1']), bias['bh1'])
    hidden_output = tf.nn.relu(hidden_out_before_activation)
    
    output = tf.add(tf.maxmul(hidden_output, weight['wo']), bias['bo'])
    return output